**Group information**

| Family name | First name | Email address |
| ----------- | ---------- | ------------- |
|             |            |               |
|             |            |               |
|             |            |               |

**Design Document**
Add your Design Document based on Lecture 1 below:


# Assignment_03_model_training_solution

Make sure to use a GPU and have access to internet connection in the Kaggle notebook:

1.  On the arrow on the bottom right, select "Notebook Options" and then "Accelerator" to choose the GPU P100, and select "Variables & Files" under Persistence. **Note that Kaggle allows 30h per week per user of accelerated computing. Plan your work accordingly. It takes time to load the data and you may experience unavailability of GPUs or Session Errors.**
1. Make sure your Kaggle account is phone verified by clicking "Get phone verified" in the left sidebar under "Notebook options" and following the steps (this step is required to switch on the internet connection needed to install packages). 
1. After phone verification, the full settings menu should be visible. Toggle the "Internet" switch.

More visualizations of the process to connect the notebook to the internet are provided [here](https://stackoverflow.com/questions/68142524/cannot-access-internet-on-kaggle-notebook)


## Requirements:

1. Downloading the tiles of the ecti2021 here: [ecti2021.zip](https://www.dropbox.com/scl/fi/tuvroadxyummourvx6cmz/ecti2021.zip?rlkey=wsc19sue84ytkphheptq28ica&st=9p8npaly&dl=0)
1. Go to the "Side Bar", Click on "Input"
1. Upload as `ecti2021` the file: `ecti2021.zip`  which contains the following four files: `train.zip`, `val_without_ref_labels.zip` , and the `water_tiles.csv` and `background_tiles.csv` from the `data_preparation.ipynb` notebook. 

# Step 0: Enviroment setting

In [ ]:
!pip install segmentation-models-pytorch
!pip install -U git+https://github.com/albu/albumentations --no-cache-di

In [ ]:
# load packages
import os
import sys
import cv2
import numpy as np
import pandas as pd
from glob import glob
import torch.nn as nn
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import segmentation_models_pytorch as smp
# Set up plotting options
%matplotlib inline
import pickle
from pickle import load
import torch
from torch.utils.data import Dataset, DataLoader

# Step 1: Load the dataset files

In [ ]:
# Set path to where dataset is downloaded
dataset_root = '/kaggle/input/ecti2021/ecti2021/ecti2021' #set accordingly based on how you uploaded the data
# get number of training/validation regions
train_dir = os.path.join(dataset_root, 'train/train')
test_dir = os.path.join(dataset_root, 'val_without_ref_labels/val')

n_train_regions = len(glob(train_dir+'/*/'))
n_test_regions = len(glob(test_dir+'/*/'))

# NOTE: make sure number of regions is NOT 0, otherwise it might be that the code is not able to read the data. 
print('Number of training temporal-regions: {}'.format(n_train_regions))
print('Number of test temporal-regions: {}'.format(n_test_regions))

From the Lab_01_data_preparation, we identified that the ETCI 2021 Competition on Flood Detection is composed of 33'405 tiles. However, we also identified tiles that have empty VV/VH but have a non-zero label. We already excluded these tiles when saving the `water_tiles.csv` and `background_tiles.csv`. The dataset used in this notebook should contain 27'214 tiles. 

## Utils functions

In [ ]:
def visualize(df_row, figsize=[25, 15]):
    # get image paths
    vv_image_path = df_row['vv_image_path']
    vh_image_path = df_row['vh_image_path']
    flood_label_path = df_row['flood_label_path']
    water_body_label_path = df_row['water_body_label_path']

    # create RGB image from S1 images
    rgb_name = get_filename(vv_image_path)
    vv_image = cv2.imread(vv_image_path, 0) / 255.0
    vh_image = cv2.imread(vh_image_path, 0) / 255.0
    rgb_image = s1_to_rgb(vv_image, vh_image)

    # get water body label mask
    water_body_label_image = cv2.imread(water_body_label_path, 0) / 255.0

    # plot images
    plt.figure(figsize=tuple(figsize))
    if df_row.isnull().sum() > 0:
        # plot RGB S1 image
        plt.subplot(1,2,1)
        plt.imshow(rgb_image)
        plt.title(rgb_name)

        # plot water body mask
        plt.subplot(1,2,2)
        plt.imshow(water_body_label_image)
        plt.title('Water body mask')
    else:
        flood_label_image = cv2.imread(flood_label_path, 0) / 255.0
        # plot RGB S1 image
        plt.subplot(1,3,1)
        plt.imshow(rgb_image)
        plt.title(rgb_name)

        # plot flood label mask
        plt.subplot(1,3,2)
        plt.imshow(flood_label_image)
        plt.title('Flood mask')

        # plot water body mask
        plt.subplot(1,3,3)
        plt.imshow(water_body_label_image)
        plt.title('Water body mask')

def s1_to_rgb(vv_image, vh_image):
    eps=1e-06
    ratio_image = np.clip(np.nan_to_num(vv_image/(vh_image+eps), 0), 0, 1) # outside [0,1] will be clipped
    rgb_image = np.stack((vv_image, vh_image, ratio_image), axis=2) #different from lab01: np.abs(red) / np.abs(green) 
    return rgb_image

def visualize_result(df_row, prediction, figsize=[25, 15]):
    vv_image = cv2.imread(df_row['vv_image_path'], 0) / 255.0
    vh_image = cv2.imread(df_row['vh_image_path'], 0) / 255.0
    rgb_input = s1_to_rgb(vv_image, vh_image)

    plt.figure(figsize=tuple(figsize))
    plt.subplot(1,2,1)
    plt.imshow(rgb_input)
    plt.title('RGB w/ result')
    plt.subplot(1,2,2)
    plt.imshow(prediction)
    plt.title('Result')

# Step 2: Create training dataframes

In [ ]:
def get_filename(filepath,split_symbol='/'):
    return filepath.split(split_symbol)[-1]

def read_csv(csvpath,split_symbol='\\'):
    path_list = np.loadtxt(csvpath,delimiter=" ", dtype=str).tolist()
    return [get_filename(pth,split_symbol) for pth in path_list]

In [ ]:
water_image_names = read_csv('/kaggle/input/ecti2021/ecti2021/ecti2021/water_tiles.csv') #from lab01 make sure the path is correct
background_image_names = read_csv('/kaggle/input/ecti2021/ecti2021/ecti2021/background_tiles.csv')

region_name_dates0 = ['_'.join(n.split('_')[:2]) for n in water_image_names]
region_name_dates1 = ['_'.join(n.split('_')[:2]) for n in background_image_names]

vv_image_paths, vh_image_paths, flood_label_paths, water_body_label_paths = [], [], [], []


water_image_paths,background_image_paths = [],[]

for i in range(len(water_image_names)):
    vv_image_path = os.path.join(train_dir, region_name_dates0[i], 'tiles', 'vv', water_image_names[i])
    vv_image_paths.append(vv_image_path)
    water_image_paths.append(vv_image_path)
    
    # get vh image path
    vh_image_name = water_image_names[i].replace('vv', 'vh')
    vh_image_path = os.path.join(train_dir, region_name_dates0[i], 'tiles', 'vh', vh_image_name)
    vh_image_paths.append(vh_image_path)

    # get flood mask path
    flood_image_name = water_image_names[i].replace('_vv', '')
    flood_label_path = os.path.join(train_dir, region_name_dates0[i], 'tiles', 'flood_label', flood_image_name)
    flood_label_paths.append(flood_label_path)

    # get water body mask path
    water_body_label_name = water_image_names[i].replace('_vv', '')
    water_body_label_path = os.path.join(train_dir, region_name_dates0[i], 'tiles', 'water_body_label', water_body_label_name)
    water_body_label_paths.append(water_body_label_path)
    
for i in range(len(background_image_names)):
    vv_image_path = os.path.join(train_dir, region_name_dates1[i], 'tiles', 'vv', background_image_names[i])
    vv_image_paths.append(vv_image_path)
    background_image_paths.append(vv_image_path)
    
    # get vh image path
    vh_image_name = background_image_names[i].replace('vv', 'vh')
    vh_image_path = os.path.join(train_dir, region_name_dates1[i], 'tiles', 'vh', vh_image_name)
    vh_image_paths.append(vh_image_path)

    # get flood mask path
    flood_image_name = background_image_names[i].replace('_vv', '')
    flood_label_path = os.path.join(train_dir, region_name_dates1[i], 'tiles', 'flood_label', flood_image_name)
    flood_label_paths.append(flood_label_path)

    # get water body mask path
    water_body_label_name = background_image_names[i].replace('_vv', '')
    water_body_label_path = os.path.join(train_dir, region_name_dates1[i], 'tiles', 'water_body_label', water_body_label_name)
    water_body_label_paths.append(water_body_label_path)
    

In [ ]:
#Shuffle the index and then split in train and validation
n=len(vv_image_paths) #number of images in the dataset
arr = np.arange(n) #array 0...n-1
np.random.shuffle(arr) # shuffle it
train_idx=arr[0:int(np.round(0.80*n))] #80% train
valid_idx=arr[int(np.round(0.80*n)):] #20% validation
print("Number of tiles in training set:",train_idx.size)
print("Number of tiles in validation set:",valid_idx.size)
print("Number of tiles in the training and validation set:",train_idx.size+valid_idx.size) 

In [ ]:
vv_image_paths_train = list(np.array(vv_image_paths)[train_idx])
vh_image_paths_train = list(np.array(vh_image_paths)[train_idx])
flood_label_paths_train = list(np.array(flood_label_paths)[train_idx])
water_body_label_paths_train = list(np.array(water_body_label_paths)[train_idx])

train_paths = {'vv_image_path': vv_image_paths_train,
        'vh_image_path': vh_image_paths_train,
        'flood_label_path': flood_label_paths_train,
        'water_body_label_path': water_body_label_paths_train,
}

train_df = pd.DataFrame(train_paths)

print(train_df.shape)
train_df.head()

In [ ]:
vv_image_paths_valid = list(np.array(vv_image_paths)[valid_idx])
vh_image_paths_valid = list(np.array(vh_image_paths)[valid_idx])
flood_label_paths_valid = list(np.array(flood_label_paths)[valid_idx])
water_body_label_paths_valid = list(np.array(water_body_label_paths)[valid_idx])

valid_paths = {'vv_image_path': vv_image_paths_valid,
        'vh_image_path': vh_image_paths_valid,
        'flood_label_path': flood_label_paths_valid,
        'water_body_label_path': water_body_label_paths_valid,
}


valid_df = pd.DataFrame(valid_paths)

print(valid_df.shape)
valid_df.head()

## # Step 2b: Create training undersampled dataframes

In [ ]:
background_image_paths_train = [path for path in background_image_paths if path in vv_image_paths_train]
background_num_train = len(background_image_paths_train)
print('Number of background tiles included in training:',background_num_train)

water_image_paths_train = [path for path in water_image_paths if path in vv_image_paths_train]
water_image_names_train = [get_filename(pth) for pth in water_image_paths_train]
region_name_dates2 = ['_'.join(n.split('_')[:2]) for n in water_image_names_train]
water_num_train = len(water_image_paths_train)
print('Number of water tiles included in training:',water_num_train)

In [ ]:
num_samples = int(water_num_train * 0.15) #include 15% of water tiles
arr = np.arange(int(water_num_train * 0.15))  # array 0...n-1
np.random.shuffle(arr)  # shuffle it
background_image_paths_train_undersampled = list(np.array(background_image_paths_train)[arr[0:num_samples]])
background_image_names_train_undersampled = [get_filename(pth) for pth in background_image_paths_train_undersampled]
print('Number of background tiles included in training after undersampling:',len(background_image_names_train_undersampled))
region_name_dates3 = ['_'.join(n.split('_')[:2]) for n in background_image_names_train_undersampled]

vh_image_paths_train_undersampled, flood_label_paths_train_undersampled, water_body_label_paths_train_undersampled = [], [], []
for i in range(len(water_image_names_train)):
    # get vh image path
    vh_image_name = water_image_names_train[i].replace('vv', 'vh')
    vh_image_path = os.path.join(train_dir, region_name_dates2[i], 'tiles', 'vh', vh_image_name)
    vh_image_paths_train_undersampled.append(vh_image_path)

    # get flood mask path
    flood_image_name = water_image_names_train[i].replace('_vv', '')
    flood_label_path = os.path.join(train_dir, region_name_dates2[i], 'tiles', 'flood_label', flood_image_name)
    flood_label_paths_train_undersampled.append(flood_label_path)

    # get water body mask path
    water_body_label_name = water_image_names_train[i].replace('_vv', '')
    water_body_label_path = os.path.join(train_dir, region_name_dates2[i], 'tiles', 'water_body_label', water_body_label_name)
    water_body_label_paths_train_undersampled.append(water_body_label_path)

vv_image_paths_train_undersampled = water_image_paths_train
print('Number of water body label included in training after undersampling:',len(water_body_label_paths_train_undersampled))
for i in range(len(background_image_names_train_undersampled)):
    vv_image_paths_train_undersampled.append(background_image_paths_train_undersampled[i])
    
    # get vh image path
    vh_image_name = background_image_names_train_undersampled[i].replace('vv', 'vh')
    vh_image_path = os.path.join(train_dir, region_name_dates3[i], 'tiles', 'vh', vh_image_name)
    vh_image_paths_train_undersampled.append(vh_image_path)

    # get flood mask path
    flood_image_name = background_image_names_train_undersampled[i].replace('_vv', '')
    flood_label_path = os.path.join(train_dir, region_name_dates3[i], 'tiles', 'flood_label', flood_image_name)
    flood_label_paths_train_undersampled.append(flood_label_path)

    # get water body mask path
    water_body_label_name = background_image_names_train_undersampled[i].replace('_vv', '')
    water_body_label_path = os.path.join(train_dir, region_name_dates3[i], 'tiles', 'water_body_label', water_body_label_name)
    water_body_label_paths_train_undersampled.append(water_body_label_path)
assert len(vv_image_paths_train_undersampled)==len(vh_image_paths_train_undersampled)==len(flood_label_paths_train_undersampled)==len(water_body_label_paths_train_undersampled)
print('Number of overall images  included in training after undersampling:',len(water_body_label_paths_train_undersampled))

**1) Based on the consideration done in Lab1 and the above calculation, explain the original dataset is in term of class imbalance and how this changed in the undersample dataset.**



**ANSWER HERE:** 

In [ ]:
train_paths_undersample = {'vv_image_path': vv_image_paths_train_undersampled,
        'vh_image_path': vh_image_paths_train_undersampled,
        'flood_label_path': flood_label_paths_train_undersampled,
        'water_body_label_path': water_body_label_paths_train_undersampled
}
train_df_undersample = pd.DataFrame(train_paths_undersample)

print(train_df_undersample.shape)
train_df_undersample.head() 

# Step 3: Visualize images

This section is meant to be used to experiment the data. Feel free to change things and to explore the data.

In [ ]:
train_df

In [ ]:
cv2.imread(train_df_undersample.iloc[1200]['vv_image_path'],0)

In [ ]:
train_df_undersample.iloc[3600]['vv_image_path']

In [ ]:
visualize(train_df_undersample.iloc[3600])

In [ ]:
visualize(train_df.iloc[3677])

# Step 4: Setup the dataset for machine learning

Since the Phase 1 (Development phase) of the ETCI 2021 Competition on Flood Detection provided with training data (which includes reference data) and a validation data (without reference data until phase 1 concludes),we will split our training dataset (that contains flood masks) into a smaller training and development set. 


### Create a PyTorch dataset

We will be using the PyTorch deep learning library to format this dataset and create our machine learning model. Therefore we will need to create a custom Dataset class and pass it into a DataLoader object (see the [PyTorch Dataset Tutorial](https://pytorch.org/tutorials/beginner/data_loading_tutorial.html)  for more detail on the topic). To compute image transformations we will use the [Albumentations](https://github.com/albumentations-team/albumentations) package. 

In [ ]:
class ETCIDataset(Dataset):
    def __init__(self, dataframe, split, transform=None):
        self.split = split
        self.dataset = dataframe
        self.transform = transform

    def __len__(self):
        return self.dataset.shape[0]


    def __getitem__(self, index):
        example = {}
        
        df_row = self.dataset.iloc[index]

        # load vv and vh images
        vv_image = cv2.imread(df_row['vv_image_path'], 0) / 255.0
        vh_image = cv2.imread(df_row['vh_image_path'], 0) / 255.0
        
        # convert vv and vh images to rgb
        rgb_image = s1_to_rgb(vv_image, vh_image)

        if self.split == 'test':
            # no flood mask should be available
            example['image'] = rgb_image.transpose((2,0,1)).astype('float32')  #HWC->CHW
        else:
            # load ground truth flood mask
            flood_mask = cv2.imread(df_row['flood_label_path'], 0) / 255.0

            # compute transformations
            
            if self.transform:
                augmented = self.transform(image=rgb_image, mask=flood_mask)
                rgb_image = augmented['image']
                flood_mask = augmented['mask']

            example['image'] = rgb_image.transpose((2,0,1)).astype('float32') #HWC->CHW
            example['mask'] = flood_mask.astype('int64')

        return example


**2) Check the [Albumentations](https://github.com/albumentations-team/albumentations) package and implement both Vertical and Horizontal flip with probability 0.5 and RandomResizedCrop of 256 on both dimentions. Then load the train and validation set.**

In [ ]:
import albumentations as A

### BEGINNING OF THE CODE
# trasform= ...
#train_dataset = ...
#valid_dataset = ...
####END OF THE CODE
print('Trainining set size:',len(train_dataset))
print('Validation set size:',len(valid_dataset))

In [ ]:
batch_size = 64

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=True)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False, num_workers=2, pin_memory=True)

In [ ]:
train_undersampled_dataset = ETCIDataset(train_df_undersample, split='train', transform=transform)
train_undersampled_loader = DataLoader(train_undersampled_dataset, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=True)
print('Undersampled Trainining set size:',len(train_undersampled_dataset))

# Step 5: Deep learning model creation

### Select hardware to train model

In [ ]:
device = 'cuda' 

**3) Read carefully the documentation of the segmentation model from the [Segmentation Models](https://github.com/qubvel/segmentation_models.pytorch) package ([documentation here](https://smp.readthedocs.io/en/latest/)),  implement a UNet with resnet34 as encoder, without any pre-trained weights, and the appropriate number of in_channel and classes based on the dataset.**

In [ ]:
def create_model():
  ###CODE HERE


In [ ]:
model_1 = create_model()
model_1.to(device) # load model into GPU memory

### Metric tracker

In [ ]:
from sklearn.metrics import confusion_matrix

class EvalTracker:
    def __init__(self, n_classes=2, smooth=0.0001):
        self.n_classes = n_classes
        self.reset()
        self.smooth = smooth

    def reset(self):
        self.cm = np.zeros((self.n_classes, self.n_classes))
        self.count = 0
    
    def update(self, pred, target):
        # pred: [B, 2, H, W]
        # target: [B, H, W]
        self.count += pred.shape[0]

        # reshape inputs
        pred = pred.argmax(dim=1).flatten()  # [B*H*W]
        target = target.flatten()  # [B*H*W]

        # put into cpu memory
        pred = pred.detach().cpu().numpy()
        target = target.detach().cpu().numpy()

        # compute confusion matrix values
        self.cm += confusion_matrix(target, pred)

    def get_mean(self):
        tn, fp, fn, tp = self.cm.ravel()

        # compute IoU
        iou = tp / (tp + fp + fn + self.smooth)
        prec = tp / (tp + fp + self.smooth)
        rec = tp / (tp + fn + self.smooth)
        f1 = 2.0*prec*rec/(prec+rec)

        return iou, prec, rec, f1


# Step 6: Train model on the full dataset

### Model config, optimizer and loss function

In [ ]:
# set the number of times you want the model to see all of the training data
epochs = 20
learning_rate = 1e-4
optimizer = torch.optim.Adam(model_1.parameters(), lr=learning_rate)
criteria_no_weights = nn.CrossEntropyLoss(weight=None)

### Training loop

In [ ]:
train_loss_dict={}
val_loss_dict={}
for epoch in range(epochs):
    print('Epoch: [{}/{}]'.format(epoch+1, epochs))

    # train set
    pbar = tqdm(train_loader)
    train_loss = 0.0
    model_1.train()
    eval_logger = EvalTracker()
    for batch in pbar:
        # load image and mask into device memory
        image = batch['image'].to(device)
        mask = batch['mask'].to(device)

        # pass images into model
        pred = model_1(image)

        # get loss
        loss = criteria_no_weights(pred, mask)

        # update the model
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
               
        # compute and display progress
        eval_logger.update(pred, mask)
        mIoU, Prec, Rec, f1 = eval_logger.get_mean()
        pbar.set_description('Loss: {0:1.4f} | mIoU {1:1.4f} | Prec {2:1.4f} | Rec {3:1.4f}  | F1 score {4:1.4f}'.format(loss.item(), mIoU, Prec, Rec, f1))
        train_loss += loss.item() * image.size(0)
    # calculate the average loss for both training and validation
    train_loss /= len(train_loader.dataset)
    train_loss_dict[epoch] = train_loss
        
    # valid set
    pbar = tqdm(valid_loader)
    model_1.eval()
    eval_logger = EvalTracker()
    val_loss = 0.0
    with torch.no_grad():
        for batch in pbar:
            # load image and mask into device memory
            image = batch['image'].to(device)
            mask = batch['mask'].to(device)

            # pass images into model
            pred = model_1(image)

            # get loss
            loss = criteria_no_weights(pred, mask)
                       
            # compute and display progress
            eval_logger.update(pred, mask)
            mIoU, Prec, Rec, f1 = eval_logger.get_mean()
            pbar.set_description('Loss: {0:1.4f} | mIoU {1:1.4f} | Prec {2:1.4f} | Rec {3:1.4f}  | F1 score {4:1.4f}'.format(loss.item(), mIoU, Prec, Rec, f1))
            val_loss += loss.item() * image.size(0)
    val_loss /= len(valid_loader.dataset)
    val_loss_dict[epoch] = val_loss
    
# Save the training loss values
with open('./train_loss_1_BCE.pkl', 'wb') as file:
    pickle.dump(train_loss_dict, file)
     
# Save the validation loss values
with open('./val_loss_1_BCE.pkl', 'wb') as file:
    pickle.dump(val_loss_dict, file)            
# save model
torch.save(model_1.state_dict(), 'model_1_BCE.pt')

### Plot Losses

In [ ]:
# Load the training and validation loss dictionaries
train_loss = load(open('/kaggle/working/train_loss_1_BCE.pkl', 'rb'))
val_loss = load(open('/kaggle/working/val_loss_1_BCE.pkl', 'rb'))
 
# Retrieve each dictionary's values
train_values = train_loss.values()
val_values = val_loss.values()
 
# Generate a sequence of integers to represent the epoch numbers
epochs_range = range(1, epochs+1)
 
# Plot and label the training and validation loss values
plt.plot(epochs_range, train_values, label='Training Loss')
plt.plot(epochs_range, val_values, label='Validation Loss')
 
# Add in a title and axes labels
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
 
# Set the tick locations
plt.xticks(np.arange(0, epochs+1, 2))
 
# Display the plot
plt.legend(loc='best')
plt.show()

# Step 7: Train model on the undersampled dataset

### Model config, optimizer and loss function

In [ ]:
model_2 = create_model()
model_2.to(device)
optimizer = torch.optim.Adam(model_2.parameters(), lr=learning_rate)
criteria_no_weights = nn.CrossEntropyLoss(weight=None)

**4) Implement a training loop similar to the one above but for the undersampled dataset. Use model_2 to avoid any overwriting of the previous model. Save the model as 'model_2d_BCE.pt'***

### Training loop

In [ ]:
### CODE HERE###

In [ ]:
train_path =r'train_df.csv'
valid_path = r'valid_df.csv'
train_under_path = r'train_df_undersample.csv'
train_df.to_csv(train_path)
valid_df.to_csv(valid_path)
train_df_undersample.to_csv(train_under_path)

### Plot Losses

In [ ]:
# Load the training and validation loss dictionaries
train_loss = load(open('/kaggle/working/train_loss_2d_BCE.pkl', 'rb'))
val_loss = load(open('/kaggle/working/val_loss_2d_BCE.pkl', 'rb'))
 
# Retrieve each dictionary's values
train_values = train_loss.values()
val_values = val_loss.values()
 
# Generate a sequence of integers to represent the epoch numbers
epochs_range = range(1, epochs+1)
 
# Plot and label the training and validation loss values
plt.plot(epochs_range, train_values, label='Training Loss')
plt.plot(epochs_range, val_values, label='Validation Loss')
 
# Add in a title and axes labels
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
 
# Set the tick locations
plt.xticks(np.arange(0, epochs+1, 2))
 
# Display the plot
plt.legend(loc='best')
plt.show()

# Step 8: Train model on the undersampled dataset with a weighted loss function

### Defining the split for the weighted Cross Entropy Loss

In [ ]:
#It take quite a long time to calcualte, the ratio is around 5% flooded pixels, 95% background
n_size = len(flood_label_paths_train_undersampled)
n_flooded = np.ndarray((n_size,),)

for i in tqdm(range(n_size)):
    flood_label=cv2.imread(flood_label_paths_train_undersampled[i], 0)
    n_flooded[i] = np.sum(flood_label)/255

n_flooded_ratio = np.divide(n_flooded,256*256)
flooded_pixels = np.sum(n_flooded)
background_pixels = 256*256*n_size-np.sum(n_flooded)
print("Flooded Pixels:", flooded_pixels)
print("Background Pixels:", background_pixels)
print("Ratio:", np.mean(n_flooded_ratio))

### Model config, optimizer and loss function

**5) Define the "Model config, optimizer and loss function" section as previously done but for "model_3" which will be trained with a [Weighted Cross Entropy Loss](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html). Remember to store the weights as a torch tensor, to load it in the GPU, and be careful on the order of your weights.**

In [ ]:
###CODE HERE

**6) Why did you choose the weights you used for the CrossEntropyLoss?**


### Training Loop

In [ ]:
train_loss_dict={}
val_loss_dict={}
for epoch in range(epochs):
    print('Epoch: [{}/{}]'.format(epoch+1, epochs))

    # train set
    pbar = tqdm(train_undersampled_loader)
    train_loss = 0.0
    model_3.train()
    eval_logger = EvalTracker()
    for batch in pbar:
        # load image and mask into device memory
        image = batch['image'].to(device)
        mask = batch['mask'].to(device)

        # pass images into model
        pred = model_3(image)

        # get loss
        loss = criteria_weights(pred, mask)

        # update the model
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # compute and display progress
        eval_logger.update(pred, mask)
        mIoU, Prec, Rec, f1 = eval_logger.get_mean()
        pbar.set_description('Loss: {0:1.4f} | mIoU {1:1.4f} | Prec {2:1.4f} | Rec {3:1.4f}  | F1 score {4:1.4f}'.format(loss.item(), mIoU, Prec, Rec, f1))
        train_loss += loss.item() * image.size(0)
    # calculate the average loss for both training and validation
    train_loss /= len(train_loader.dataset)
    train_loss_dict[epoch] = train_loss
    # valid set
    pbar = tqdm(valid_loader)
    val_loss = 0.0
    model_3.eval()
    eval_logger = EvalTracker()
    with torch.no_grad():
        for batch in pbar:
            # load image and mask into device memory
            image = batch['image'].to(device)
            mask = batch['mask'].to(device)

            # pass images into model
            pred = model_3(image)

            # get loss
            loss = criteria_weights(pred, mask)
            
            # compute and display progress
            eval_logger.update(pred, mask)
            mIoU, Prec, Rec, f1 = eval_logger.get_mean()
            pbar.set_description('Loss: {0:1.4f} | mIoU {1:1.4f} | Prec {2:1.4f} | Rec {3:1.4f}  | F1 score {4:1.4f}'.format(loss.item(), mIoU, Prec, Rec, f1))
            val_loss += loss.item() * image.size(0)
    val_loss /= len(valid_loader.dataset)
    val_loss_dict[epoch] = val_loss
# Save the training loss values
with open('./train_loss_2d_WBCE.pkl', 'wb') as file:
    pickle.dump(train_loss_dict, file)
     
# Save the validation loss values
with open('./val_loss_2d_WBCE.pkl', 'wb') as file:
    pickle.dump(val_loss_dict, file)   
# save model
torch.save(model_3.state_dict(), 'model_2d_WBCE.pt')

### Plot Losses

In [ ]:
from numpy import *
from pickle import load
# Load the training and validation loss dictionaries
train_loss = load(open('/kaggle/working/train_loss_2d_WBCE.pkl', 'rb'))
val_loss = load(open('/kaggle/working/val_loss_2d_WBCE.pkl', 'rb'))
 
# Retrieve each dictionary's values
train_values = train_loss.values()
val_values = val_loss.values()
 
# Generate a sequence of integers to represent the epoch numbers
epochs_range = range(1, epochs+1)
 
# Plot and label the training and validation loss values
plt.plot(epochs_range, train_values, label='Training Loss')
plt.plot(epochs_range, val_values, label='Validation Loss')
 
# Add in a title and axes labels
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
 
# Set the tick locations
plt.xticks(np.arange(0, epochs+1, 2))
 
# Display the plot
plt.legend(loc='best')
plt.show()

**7) How are the three models (model_1_BCE.pt, model_2d_BCE.pt, and model_2d_WBCE.pt) performning? Comment the performances of the models.**

# Step 9a (Optional): Train more models
Feel free to train mode model changing the configuration, learning rate, optimizer, loss function, etc. This is fully optional and bonus points will be assigned to go beyond the max grade. Total freedom here to create #Step 9b, 9c, etc.. Rememebr that this is 100% optional and won't influence your grading, it is a space to experiment ideas. I suggest you finalize the notebook first given the limited computational resources and get back to this later. 

Please add in the text cell below the idea behind the experiment you are about to run. Why do you want to test that specific conifguration? What do you expect in terms of results and what did you get after training? 

In [ ]:
### CREATE A TEXT CELL BELOW AS EXPLAINED ABOVE EXPLAINING YOUR EXPERIMENT

Let's train the second model with transfer learning from Imagenet using imagenet stats

#### Create the dataloader (if needed) and the model (if needed)

In [ ]:
### CODE HERE


### Model config, optimizer and loss function

In [ ]:
###CODE HERE


### Training Loop

In [ ]:
### CODE HERE

### Plot Losses

In [ ]:
### CODE HERE


# Step 10: Test models

### Create a test dataset
Let's create Dataset and DataLoader objects for the validation set. This time we will not have labels.

In [ ]:
vv_image_paths = sorted(glob(test_dir+'/**/vv/*.png', recursive=True))
vv_image_names = [get_filename(pth) for pth in vv_image_paths]
region_name_dates = ['_'.join(n.split('_')[:2]) for n in vv_image_names]

vh_image_paths, flood_label_paths, water_body_label_paths, region_names = [], [], [], []
for i in range(len(vv_image_paths)):
    # get vh image path
    vh_image_name = vv_image_names[i].replace('vv', 'vh')
    vh_image_path = os.path.join(test_dir, region_name_dates[i], 'tiles', 'vh', vh_image_name)
    vh_image_paths.append(vh_image_path)

    # get flood mask path ()
    flood_label_paths.append(np.NaN)

    # get water body mask path
    water_body_label_name = vv_image_names[i].replace('_vv', '')
    water_body_label_path = os.path.join(test_dir, region_name_dates[i], 'tiles', 'water_body_label', water_body_label_name)
    water_body_label_paths.append(water_body_label_path)

    # get region name
    region_name = region_name_dates[i].split('_')[0]
    region_names.append(region_name)

test_paths = {'vv_image_path': vv_image_paths,
        'vh_image_path': vh_image_paths,
        'flood_label_path': flood_label_paths,
        'water_body_label_path': water_body_label_paths,
        'region': region_names
}

test_df = pd.DataFrame(valid_paths)

print(test_df.shape)
test_df.head()

### Run inference

**8) Choose your best performing model from Steps 6-9 and run inference below:**

In [ ]:
# load model
model_test = create_model()
### CODE HERE

### Visualize some results

In [ ]:
index = 252
visualize_result(valid_df.iloc[index], final_predictions[index], figsize=(17,10))

In [ ]:
index = -100
visualize_result(valid_df.iloc[index], final_predictions[index], figsize=(17,10))

In [ ]:
index = 1910
visualize_result(valid_df.iloc[index], final_predictions[index], figsize=(17,10))